In [559]:
import pandas as pd
import re

from preprocessing.data_cleaning import DataCleaner

pd.options.display.max_columns = 999

In [560]:
# LEITURA DOS DADOS BRUTOS ADQUIRIDOS 

df_raw_ = pd.read_parquet("C:\\Users\\Dudu_\\OneDrive\\Documentos\\Estudos\\1. Projetos Pessoais\\3. Previsão Público Cruzeiro\\data\\refined\\df_refined.parquet",use_pandas_metadata=False, engine="pyarrow")

df = df_raw_.copy()

In [561]:
df.shape

(926, 23)

In [562]:
x = (df.isna().sum() )
x.sort_values(ascending=False).round(2)

publico_presente                   271
renda_bruta                         54
publico_pagante                     35
adversario                           0
ano                                  0
dia_semana                           0
estadio                              0
campeonato                           0
placar                               0
horario                              0
dia                                  0
mes                                  0
data_formatada                       0
campeonato_ajustado                  0
gols_visitante                       0
gols_mandante                        0
gols_total                           0
pontos_alcancados                    0
resultado                            0
aproveitamento_temporada_previo      0
jogos_invencibilidade_previo         0
estreia                              0
classico                             0
dtype: int64

In [563]:
df[['gols_visitante','gols_mandante','dia','mes']] = df[['gols_visitante','gols_mandante','dia','mes']].astype(int)

In [564]:
df['resultado_previo'] = df['resultado'].shift(1)
df['gols_mandante_previo'] = df['gols_mandante'].shift(1)
df['gols_visitante_previo'] = df['gols_visitante'].shift(1)
df['gols_total_previo'] = df['gols_total'].shift(1)
df['saldo_gols_previo'] = df['gols_mandante_previo'] - df['gols_visitante_previo']

def periodo_do_dia(horario):
    hora = int(horario.split(':')[0])
    if hora < 12:
        return 'manha'
    elif hora < 18:
        return 'tarde'
    else:
        return 'noite'

df['periodo'] = df['horario'].apply(periodo_do_dia)

In [565]:
df = df[df['adversario'] != 'Cruzeiro']

In [566]:
df = df.drop(df[df.publico_pagante.isna()].index, axis=0)

df = df[~(df['publico_pagante'] == 0)]

In [567]:
df['publico_pagante_ultimo_jogo_em_casa'] = df['publico_pagante'].shift(1)
df['renda_bruta_ultimo_jogo_em_casa'] = df['renda_bruta'].shift(1)

df = df.drop(df[df['publico_pagante_ultimo_jogo_em_casa'].isna()].index, axis=0)

In [568]:
df = df.drop(['publico_presente','placar','campeonato','data_formatada','renda_bruta','gols_visitante','gols_mandante','gols_total','pontos_alcancados','resultado', 'ano','adversario'], axis=1)

df = df[df['campeonato_ajustado'] == "Brasileirão"]
df = df.drop(['campeonato_ajustado'], axis=1)

In [569]:
df = df.reset_index(drop=True)

In [570]:
df = pd.get_dummies(df, drop_first=True)

### ANÁLISE DAS MELHORES VARIAVEIS

In [571]:
linear_dep = pd.DataFrame()

print("Quantidade de colunas numéricas restantes", len(df.select_dtypes(include='number').columns))

print("Quantidade de colunas categóricas restantes", len(df.select_dtypes(exclude='number').columns))

num_df = df[df.select_dtypes(include='number').columns]
num_df.fillna(0, inplace=True)

for col in num_df.columns:
    linear_dep.loc[col, 'pearson_corr'] = num_df[col].corr(num_df['publico_pagante'])
linear_dep['abs_pearson_corr'] = abs(linear_dep['pearson_corr'])

from sklearn.feature_selection import f_classif
for col in num_df.columns:
    mask = num_df[col].notnull()
    (linear_dep.loc[col, 'F'], linear_dep.loc[col, 'p_value']) = f_classif(pd.DataFrame(num_df.loc[mask, col]), num_df['publico_pagante'].loc[mask])

Quantidade de colunas numéricas restantes 13
Quantidade de colunas categóricas restantes 35


C:\Users\Dudu_\AppData\Local\Temp\ipykernel_9816\537204547.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df.fillna(0, inplace=True)
c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\numpy\lib\_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\numpy\lib\_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:109: RuntimeWarning: in

In [572]:
linear_dep.sort_values(by='abs_pearson_corr', ascending=False)

,pearson_corr,abs_pearson_corr,F,p_value
publico_pagante,1.000000,1.000000,NaN,NaN
publico_pagante_ultimo_jogo_em_casa,0.342465,0.342465,NaN,NaN
renda_bruta_ultimo_jogo_em_casa,0.256531,0.256531,NaN,NaN
classico,0.244219,0.244219,NaN,NaN
jogos_invencibilidade_previo,-0.170777,0.170777,NaN,NaN
dia,-0.134598,0.134598,NaN,NaN
mes,0.120130,0.120130,NaN,NaN
saldo_gols_previo,0.100313,0.100313,NaN,NaN
gols_visitante_previo,-0.088343,0.088343,NaN,NaN
gols_mandante_previo,0.060689,0.060689,NaN,NaN


In [573]:
linear_dep[linear_dep['p_value'] < 0.05].sort_values(by='abs_pearson_corr', ascending=False)

,pearson_corr,abs_pearson_corr,F,p_value


In [574]:
from sklearn.feature_selection import f_classif

# Select categorical columns (excluding 'publico_pagante')
cat_cols = df.select_dtypes(exclude='number').columns

anova_cat_corr = pd.DataFrame(columns=['column', 'F_value', 'p_value'])

for col in cat_cols:
    # Only use rows without missing values in both columns
    mask = df[col].notnull() & df['publico_pagante'].notnull()
    if mask.sum() > 1 and df[col][mask].nunique() > 1:
        X = pd.DataFrame(df[col][mask])
        y = df['publico_pagante'][mask]
        # Convert categorical to codes for ANOVA
        X_encoded = X.apply(lambda x: x.astype('category').cat.codes)
        F, p = f_classif(X_encoded, y)
        anova_cat_corr = pd.concat([anova_cat_corr, pd.DataFrame({'column': [col], 'F_value': [F[0]], 'p_value': [p[0]]})], ignore_index=True)

anova_cat_corr.sort_values('F_value', ascending=False, inplace=True)
anova_cat_corr.reset_index(drop=True, inplace=True)
anova_cat_corr[anova_cat_corr['p_value'] < 0.05].sort_values('F_value', ascending=False)

c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:109: RuntimeWarning: divide by zero encountered in divide
  msw = sswn / float(dfwn)
C:\Users\Dudu_\AppData\Local\Temp\ipykernel_9816\3530545213.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  anova_cat_corr = pd.concat([anova_cat_corr, pd.DataFrame({'column': [col], 'F_value': [F[0]], 'p_value': [p[0]]})], ignore_index=True)
c:\Users\Dudu_\OneDrive\Documentos\Estudos\1. Projetos Pessoais\3. Previsão Público Cruzeiro\cruzeiroenv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:109: RuntimeWarning: divide by zero encountered in divide
  msw = ssw

,column,F_value,p_value


In [575]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

X = df.drop('publico_pagante', axis=1)
y = df['publico_pagante'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")


MAE: 8631.141402264031
MSE: 133418481.42972977
RMSE: 11550.691816065813
R²: 0.23236113786697388


In [578]:
import matplotlib.pyplot as plt
import seaborn as sns


df_importance = pd.DataFrame(data=model.feature_importances_, index=X.columns, columns=['importance']).sort_values(by='importance', ascending=False)

df_importance['importance'].sum()

np.float32(1.0000002)